In [1]:
from selenium import webdriver as wd
import chromedriver_py
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [2]:
wd= wd.Chrome()

In [3]:
def getPandaDfForSearchResults(searchResultsPage):
    rows=[]
    for result in searchResultsPage:
        title = result.find("span", {"class": "a-text-normal"})
        # sponsored = result.find("span", {"class": "a-color-base"}, text="Sponsored")
        price = result.find("span", {"class": "a-price-whole"})
        url = result.find("a", {"class": "a-link-normal"})
        if price:
            row= [title.text, price.text, "https://amazon.com/" + url['href']]
            rows.append(row)
    
    df=pd.DataFrame.from_records(rows, columns=["Title", "Price", "URL"])
    return df

In [4]:
def getSearchResultsForPage(url):
    wd.get(url)
    time.sleep(5)
    soup = BeautifulSoup(wd.page_source)
    searchResultsPage = soup.findAll("div", {"data-component-type": "s-search-result"})
    df = getPandaDfForSearchResults(searchResultsPage)
    if list(df['Title']):
        final_list = [list(df['Title'])[0],list(df['Price'])[0],list(df['URL'])[0]]
    else:
        return []
    return final_list

In [5]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)



def start(update: Update, context: CallbackContext):
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'ሰላም {user.mention_markdown_v2()}\! , መግዛት የምትፈልጉትን ዕቃ ያስገቡ።',
        reply_markup=ForceReply(selective=True),
    )

API_KEY = 'eccbe927996c560fb653946e'
data = requests.get('https://v6.exchangerate-api.com/v6/'+API_KEY+'/latest/USD').json()
value= data['conversion_rates']['ETB']
hi_message = ['hi','Hi','hey','Hey','selam','Selam']
def echo(update: Update, context: CallbackContext):
    if update.message.text in hi_message :
        update.message.reply_text('Hi')
    elif update.message.text == 'ሰላም':
        update.message.reply_text('ሰላም')
    elif update.message.text:
        all_text = getSearchResultsForPage("https://www.amazon.com/s?k="+update.message.text)
        if not all_text:
            update.message.reply_text('በዚህ ስም ዋጋዉ የተለጠፈበት እቃ የለም።  እባክዎ በሌላ ስም ይሞክሩ።')
        else:
            update.message.reply_text(all_text[0]+'\n\n'+'ዋጋ = $'+all_text[1][:-1]+' ወይም '+str(int(all_text[1][:-1])*value)+' ብር\n\n'+all_text[2])
def main():
    updater = Updater("5047156256:AAFQUHnCMhsEG5y2LH5svRkmpJQezBnKUzY")
    dispatcher = updater.dispatcher
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
    updater.start_polling()
    updater.idle()


if __name__ == '__main__':
    main()

2022-02-06 06:16:54,726 - apscheduler.scheduler - INFO - Scheduler started
